In [130]:
from mesa import Agent, Model
from mesa.time import RandomActivation, BaseScheduler
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Polygon, Point, MultiPoint
import math
import matplotlib.pyplot as plt
import numpy as np
from numpy import exp
from numpy.random import rand, seed
import folium
import random 
from haversine import haversine
pd.options.mode.chained_assignment = None  # default='warn'

In [183]:
class ConstructionSite(Agent): 
    def __init__(self, unique_id, model, buildingType, coords, inA10):
        super().__init__(unique_id, model)
        self.buildingType = buildingType # A, B, C...etc. 
        self.coords = coords 
        self.inA10 = inA10 # True or False 
        self.material_request = dict.fromkeys(self.model.material_list, 0)
        self.materials_received = dict.fromkeys(self.model.material_list, 0)
        
        self.calc_materials_required()
            
    def calc_materials_required(self): 
        materials_required = {}
        b = self.model.build_info.copy()
        b = b[(b.buildingType == self.buildingType) & 
              (b.biobased == self.model.biobased_type)] 
        for strucType in ['structural', 'nonstructural']:
            b1 = b[b.strucType == strucType]
            for mat in self.model.material_list: 
                materials_required[f'{strucType}_{mat}'] = b1[mat].iloc[0]
        materials_required['modules'] = b.modules.iloc[0]
        self.materials_required = materials_required
        
    def step(self): 
        # if materials_received < materials_required: 
            # self.request_materials() 
        self.request_materials()
    
    def check_materials_required(self): 
        # 
        None
    
    def request_materials(self):
        material_request = {}
        if self.model.modularity_type == 'none': # struc and nonstruc
            for mat in self.model.material_list: 
                amount_struc = self.materials_required[f'structural_{mat}']
                amount_nonStruc = self.materials_required[f'nonstructural_{mat}']
                material_request[mat] = amount_struc + amount_nonStruc
        else: # structural and modules
            for mat in self.model.material_list: 
                material_request[mat] = self.materials_required[f'structural_{mat}']
            material_request['modules'] = self.materials_required['modules']
        
        for mat, requested_amount in material_request.items(): 
            material_request[mat] = round(requested_amount * 0.2)
        self.material_request = material_request

class Hub(Agent):
    def __init__(self, unique_id, model, hubType, coords, inA10, nearest_macroHub):
        super().__init__(unique_id, model)
        self.hubType = hubType # macro or micro 
        self.coords = coords
        self.inA10 = inA10
        self.nearest_macroHub = nearest_macroHub
        self.mat_to_send = {}
        self.nTrips = {}

    def step(self):
        self.find_assigned_sites()
        if self.assigned_sites: # if assigned_sites list is not empty:
            self.calc_mat_to_send() # calculate material to send to each site
            self.calc_n_trips() # calculate number of trips required for truck
            self.move_materials()
        
    def find_assigned_sites(self): 
        sites = self.model.construction_sites
        assigned_sites = [site for site in sites if site.nearestHub_id == self.unique_id]
        self.assigned_sites = assigned_sites
        
    def calc_mat_to_send(self): 
        sites = self.assigned_sites
        mat_to_send = {}
        for site in sites: 
            if sum(site.material_request.values()) > 0: 
                mat_to_send[site.unique_id] = {}
                for mat in site.material_request.keys():
                    mat_amount = site.material_request[mat]
                    mat_to_send[site.unique_id][mat] = mat_amount
        self.mat_to_send = mat_to_send
    
    def calc_n_trips(self): 
        trucks = self.model.trucks_toSite
        truck = [truck for truck in trucks if truck.hub_id == self.unique_id][0]
        for dest_id, mat_amounts in self.mat_to_send.items(): 
            nTrips_list = []
            for mat in self.model.material_list: 
                capacity = truck.capacity[mat]
                nTrips = math.ceil(mat_amounts[mat] / capacity)
                nTrips_list.append(nTrips)
            self.nTrips[dest_id] = sum(nTrips_list)
    
    def move_materials(self): 
        od = self.model.od_matrix_h2c
        trucks = self.model.trucks_toSite
        truck = [truck for truck in trucks if truck.hub_id == self.unique_id][0]
        emissions_perKm = truck.emissions_perKm 
        emissions_list = []
        for dest_id, mat_amounts in self.mat_to_send.items(): 
            row = od[(od[:,0] == dest_id) & (od[:,1] == self.unique_id)]
            distance = row[0][2]
            emissions = distance * self.nTrips[dest_id] * emissions_perKm
            emissions_list.append(emissions)
            sites = self.model.construction_sites
            dest = [site for site in sites if site.unique_id == dest_id][0]
            for mat in self.model.material_list: 
                dest.materials_received[mat] += self.mat_to_send[dest_id][mat]
        truck.emissions += sum(emissions_list)
                
class Truck_toSite(Agent):
    def __init__(self, unique_id, model, hub_id, truck_type, capacity, emissions_perKm):
        super().__init__(unique_id, model)
        self.emissions = 0
        self.hub_id = hub_id
        self.truck_type = truck_type
        self.capacity = capacity
        self.emissions_perKm = emissions_perKm
        
    def step(self):
        pass
    
class Vehicle_toSupplier(Agent): 
    def __init__(self, unique_id, model, hub_id, network_type, capacity, emissions_perKm): 
        super().__init__(unique_id, model)
        self.emissions = 0 
        self.hub_id = hub_id
        self.network_type = network_type
        self.capacity = capacity
        self.emissions_perKm = emissions_perKm
        
    def step(self): 
        pass 

In [215]:
from mesa import Model
class Model(Model):
    def __init__(self, parameters_dict): 
        '''create construction sites, hubs, and vehicles'''
        super().__init__()
        self.load_data()
        self.add_parameters(parameters_dict) 
        
        self.id_count = 0
        self.create_constructionSites()
        self.create_hubs()
        self.create_vehicles()
        
        self.create_od_matrix_h2c()
        self.create_od_matrix_h2h()
        self.assign_hubs_to_sites()
                    
    def load_data(self): 
        self.schedule = BaseScheduler(self)
        self.construction_sites_df = gpd.read_file('data/data_cleaned/construction_sites.shp')
        self.hubs_df = gpd.read_file('data/data_cleaned/hubs.shp')
        self.vehicles_info = pd.read_csv('data/data_cleaned/vehicles_info.csv')
        self.build_info = pd.read_csv('data/data_cleaned/buildingType_info.csv')
        self.material_list = ['timber', 'concrete']
        
        self.construction_sites = []
        self.hubs = []
        self.trucks_toSite = []
        self.vehicles_toSupplier = []
        
    def add_parameters(self, parameters_dict): 
        self.network_type = parameters_dict['network_type']
        self.truck_type = parameters_dict['truck_type']
        self.biobased_type = parameters_dict['biobased_type']
        self.modularity_type = parameters_dict['modularity_type']
    
    def create_constructionSites(self): 
        for i, row in self.construction_sites_df.iterrows(): 
            coords = (row.geometry.y, row.geometry.x)
            site = ConstructionSite(self.id_count, self, row.buildType, 
                                    coords, row.inA10)
            self.schedule.add(site)
            self.construction_sites.append(site)
            self.id_count += 1 
    
    def create_hubs(self): 
        for i, row in self.hubs_df.iterrows(): 
            coords = (row.geometry.y, row.geometry.x)
            hub = Hub(self.id_count, self, row.hub_type, coords, 
                                  row.inA10, row.nearMacro)
            self.schedule.add(hub)
            self.hubs.append(hub)
            self.id_count += 1 
    
    def create_vehicles(self): 
        hub_ids = [hub.unique_id for hub in self.hubs]
        
        def get_capacity(network_type, truck_type): 
            v = self.vehicles_info.copy()
            v = v[(v.transType == network_type) & (v.vehicleType == truck_type)]
            capacity_dict = {}
            for mat in self.material_list + ['modules']: 
                capacity = v[f'capacity_{mat}'].iloc[0]
                capacity_dict[mat] = capacity
            emissions_perKm = v.emissions_perKm.iloc[0]
            return capacity_dict, emissions_perKm
        
        # add truck that goes between hub and site
        for hub_id in hub_ids: 
            capacity_dict, emissions_perKm = get_capacity('road', self.truck_type)
            truck_toSite = Truck_toSite(
                self.id_count, self, hub_id, self.truck_type, 
                capacity_dict, emissions_perKm)
            self.schedule.add(truck_toSite)
            self.trucks_toSite.append(truck_toSite)
            self.id_count += 1 

        # add vehicle that goes between hub and supplier
        for hub_id in hub_ids: 
            capacity_dict, emissions_perKm = get_capacity(self.network_type, self.network_type)
            vehicle_toSupplier = Vehicle_toSupplier(
                self.id_count, self, hub_id, self.network_type, 
                capacity_dict, emissions_perKm)
            self.schedule.add(vehicle_toSupplier)  
            self.vehicles_toSupplier.append(vehicle_toSupplier)
            self.id_count += 1 
    
    def create_od_matrix_h2c(self): 
        self.od_matrix_h2c = []
        for site in self.construction_sites: 
            for hub in self.hubs: 
                distance = haversine(site.coords, hub.coords)
                self.od_matrix_h2c.append([site.unique_id, hub.unique_id, distance])
        self.od_matrix_h2c = np.array(self.od_matrix_h2c)
    
    def create_od_matrix_h2h(self): 
        self.od_matrix_h2h = []
        macroHubs = self.hubs[:2]
        for macroHub in macroHubs: 
            for hub in self.hubs: 
                distance = haversine(macroHub.coords, hub.coords)
                self.od_matrix_h2h.append([macroHub.unique_id, hub.unique_id, distance])
        self.od_matrix_h2h = np.array(self.od_matrix_h2h)
        
    def assign_hubs_to_sites(self):
        od = self.od_matrix_h2c
        for site in self.construction_sites: 
            site_od = od[od[:, 0] == site.unique_id]
            site.nearestHub_id = int(site_od[np.argmin(site_od[:, 2]), 1])
            site.nearestHub_dist = site_od[np.argmin(site_od[:, 2]), 2]
            site_od_macro = site_od[:2]
            site.nearestMacroHub_id = int(site_od_macro[np.argmin(site_od_macro[:, 2]), 1])
            site.nearestMacroHub_dist = site_od_macro[np.argmin(site_od_macro[:, 2]), 2]
            
    def step(self):
        '''make one step with model'''
        self.schedule.step()
    
    def visualize(self): 
        self.display_total_emissions()
        
        m = folium.Map([52.377231, 4.899288], zoom_start=12, tiles='cartodbdark_matter')
        self.plotLines_h2c(m, model.hubs) 
        self.plotPoints(m, model.hubs, 'red')
        self.plotPoints(m, model.construction_sites, 'white')
        return m 
    
    def display_total_emissions(self): 
        emissions_list = [truck.emissions for truck in self.trucks_toSite]
        total_emissions = round(sum(emissions_list), 2)
        print(f'total emissions: {total_emissions} tCO2eq')
    
    def plotPoints(self, m, agent_list, color): 
        for coord in [agent.coords for agent in agent_list]: 
            folium.CircleMarker(
                location=coord, radius=1, color=color
            ).add_to(m)

    def plotLines_h2c(self, m, hub_list): 
        for hub in hub_list: 
            coords_hub = hub.coords
            coords_sites = [list(site.coords) for site in hub.assigned_sites]
            for coords_site in coords_sites: 
                folium.PolyLine(
                    locations=[coords_site, coords_hub], weight=1, 
                    color='grey', dash_array='5'
                ).add_to(m)


In [217]:
parameters_dict = {
    'network_type': 'roadWater', 
    'truck_type': 'diesel', 
    'biobased_type': 'full', 
    'modularity_type': 'none'
}
model = Model(parameters_dict)
for i in range(5): 
    model.step()
model.visualize()

total emissions: 14.39 tCO2eq
